In [1]:
import psycopg2
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from geoalchemy2 import Geometry, WKTElement

In [2]:
zip_shapefile = gpd.read_file("/Users/codyschank/Dropbox/Insight/Zip_Codes/tl_2017_us_zcta510.shp", encoding = 'utf-8')
#project to geographic coordinates to match openaddresses
zip_shapefile = zip_shapefile.to_crs({'init': 'epsg:4326'})
zip_shapefile.columns = map(str.lower, zip_shapefile.columns)

In [2]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
dbname = 'map_the_vote'
username = 'codyschank' # change this to your username

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

postgres://codyschank@localhost/map_the_vote
True


In [4]:
# I don't know why this is necessary, but it is 
zip_shapefile['geom'] = zip_shapefile['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
zip_shapefile.drop('geometry', 1, inplace=True)

In [13]:
table_name = "zip5_us"
#had to run this from command ALTER TABLE zip5_us ALTER COLUMN geom SET DATA TYPE geometry;
zip_shapefile.head(0).to_sql(table_name, engine, if_exists='replace', index=False, 
                                dtype={'geom': Geometry('Polygon', srid= 4326)})

In [14]:
sql_query = """
ALTER TABLE zip5_us ALTER COLUMN geom SET DATA TYPE geometry;
"""
engine.execute(sql_query)

In [15]:
table_name = "zip5_us"
#had to run this from command ALTER TABLE zip5_us ALTER COLUMN geom SET DATA TYPE geometry;
zip_shapefile.head(1000).to_sql(table_name, engine, if_exists='append', index=False, 
                                dtype={'geom': Geometry('Polygon', srid= 4326)})

In [16]:
chunk_size = 1000
for i in range(1000, zip_shapefile.shape[0]+chunk_size, chunk_size):
    zip_shapefile[i:(i+chunk_size)].to_sql(table_name, engine, if_exists='append', index=False, 
                                            dtype={'geom': Geometry('Polygon', srid= 4326)})